Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [5]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [ ]:
train_dataset = train_dataset[]

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

---
Logistic


In [18]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
regularization_beta = 5e-3

graph = tf.Graph()
with graph.as_default():
  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  # l2 regularization
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
                        + regularization_beta * tf.nn.l2_loss(weights))

  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
 
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


import time
num_steps = 801

def accuracy(predictions, labels):
  # np.argmax(predictions, 1) ==> the biggest probability value's index
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

start = time.clock()
with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 200 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
print('%f sec' % (time.clock()- start))

Initialized
Loss at step 0: 33.071976
Training accuracy: 7.4%
Validation accuracy: 9.2%
Loss at step 200: 9.322168
Training accuracy: 74.1%
Validation accuracy: 72.4%
Loss at step 400: 5.049736
Training accuracy: 77.8%
Validation accuracy: 75.3%
Loss at step 600: 2.887670
Training accuracy: 80.5%
Validation accuracy: 77.8%
Loss at step 800: 1.787876
Training accuracy: 82.8%
Validation accuracy: 79.4%
Test accuracy: 86.7%
5.501388 sec


---
neural network


In [19]:
hidden_layer_size = 1024
batch_size = 128
regularization_beta = 5e-3

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  # variables to hidden layer
  weights_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

  #output layer
  weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)
  logits = tf.matmul(logits,  weights_output) + biases_output
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
                        + regularization_beta * (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights_output)))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_output) + biases_output)
  test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,  weights_hidden) + biases_hidden), weights_output) + biases_output)


num_steps = 3001

start = time.clock()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print('%f sec' % (time.clock()- start))

Initialized
Minibatch loss at step 0: 1920.047363
Minibatch accuracy: 4.7%
Validation accuracy: 37.9%
Minibatch loss at step 500: 126.780571
Minibatch accuracy: 84.4%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 11.041368
Minibatch accuracy: 83.6%
Validation accuracy: 85.1%
Minibatch loss at step 1500: 1.626143
Minibatch accuracy: 82.0%
Validation accuracy: 85.1%
Minibatch loss at step 2000: 0.678809
Minibatch accuracy: 86.7%
Validation accuracy: 85.1%
Minibatch loss at step 2500: 0.679698
Minibatch accuracy: 84.4%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 0.696084
Minibatch accuracy: 81.2%
Validation accuracy: 84.4%
Test accuracy: 91.2%
24.511893 sec


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
neural network

In [34]:
# set the offset to be 0 to restrict the trainning data

hidden_layer_size = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  # variables to hidden layer
  weights_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

  #output layer
  weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)
  logits = tf.matmul(logits,  weights_output) + biases_output
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_output) + biases_output)
  test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,  weights_hidden) + biases_hidden), weights_output) + biases_output)


num_steps = 3001

start = time.clock()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = 0
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print('%f sec' % (time.clock()- start))

Initialized
Minibatch loss at step 0: 452.296021
Minibatch accuracy: 10.9%
Validation accuracy: 31.0%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 61.8%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 61.8%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 61.8%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 61.8%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 61.8%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 61.8%
Test accuracy: 68.0%
22.089627 sec


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
neural network with l2 regularization

In [28]:
# set the offset to be 0 to restrict the trainning data

hidden_layer_size = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  # variables to hidden layer
  weights_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

  #output layer
  weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)
  logits = tf.matmul(logits,  weights_output) + biases_output
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
                        + regularization_beta * (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights_output)))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_output) + biases_output)
  test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,  weights_hidden) + biases_hidden), weights_output) + biases_output)


num_steps = 3001

start = time.clock()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = 0
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print('%f sec' % (time.clock()- start))

Initialized
Minibatch loss at step 0: 1908.031616
Minibatch accuracy: 7.8%
Validation accuracy: 29.6%
Minibatch loss at step 500: 128.956223
Minibatch accuracy: 100.0%
Validation accuracy: 59.4%
Minibatch loss at step 1000: 10.583728
Minibatch accuracy: 100.0%
Validation accuracy: 66.6%
Minibatch loss at step 1500: 0.964983
Minibatch accuracy: 100.0%
Validation accuracy: 69.8%
Minibatch loss at step 2000: 0.176225
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 2500: 0.107172
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 3000: 0.098966
Minibatch accuracy: 100.0%
Validation accuracy: 70.1%
Test accuracy: 76.6%
24.755958 sec


---
neural network with dropout on the hidden layer

In [31]:
# set the offset to be 0 to restrict the trainning data

hidden_layer_size = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  keep_prob = tf.placeholder(tf.float32)

  # Variables.
  # variables to hidden layer
  weights_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

  #output layer
  weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)
  logits = tf.nn.dropout(logits, keep_prob)
  logits = tf.matmul(logits,  weights_output) + biases_output
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_output) + biases_output)
  test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,  weights_hidden) + biases_hidden), weights_output) + biases_output)


num_steps = 3001

start = time.clock()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = 0
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print('%f sec' % (time.clock()- start))

Initialized
Minibatch loss at step 0: 514.497070
Minibatch accuracy: 9.4%
Validation accuracy: 29.3%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.9%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 67.2%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.6%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 66.5%
Test accuracy: 72.5%
25.126683 sec


---
neural network with l2 regularization & dropout on the hidden layer

In [32]:
# set the offset to be 0 to restrict the trainning data

hidden_layer_size = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  keep_prob = tf.placeholder(tf.float32)

  # Variables.
  # variables to hidden layer
  weights_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
  biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

  #output layer
  weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)
  logits = tf.nn.dropout(logits, keep_prob)
  logits = tf.matmul(logits,  weights_output) + biases_output
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
                        + regularization_beta * (tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights_output)))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden) + biases_hidden), weights_output) + biases_output)
  test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset,  weights_hidden) + biases_hidden), weights_output) + biases_output)


num_steps = 3001

start = time.clock()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = 0
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print('%f sec' % (time.clock()- start))

Initialized
Minibatch loss at step 0: 2123.967773
Minibatch accuracy: 10.2%
Validation accuracy: 37.4%
Minibatch loss at step 500: 129.238968
Minibatch accuracy: 100.0%
Validation accuracy: 67.6%
Minibatch loss at step 1000: 10.594511
Minibatch accuracy: 100.0%
Validation accuracy: 68.4%
Minibatch loss at step 1500: 0.969336
Minibatch accuracy: 100.0%
Validation accuracy: 69.9%
Minibatch loss at step 2000: 0.183648
Minibatch accuracy: 100.0%
Validation accuracy: 70.3%
Minibatch loss at step 2500: 0.113656
Minibatch accuracy: 100.0%
Validation accuracy: 70.5%
Minibatch loss at step 3000: 0.109173
Minibatch accuracy: 100.0%
Validation accuracy: 70.4%
Test accuracy: 76.6%
26.851626 sec


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [81]:
# set the offset to be 0 to restrict the trainning data

hidden_layer_num = 2
hidden_layer_size = 1024
batch_size = 128
regularization_beta = 1e-3
keep_probability = 0.5

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  keep_prob = tf.placeholder(tf.float32)

  # Variables.
  # input layer
  weights_input = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size],
                                                  stddev = 1 / math.sqrt(float(image_size * image_size))))
  biases_input = tf.Variable(tf.zeros([hidden_layer_size]))

  # hidden layer 
  weights_hidden = [tf.Variable(tf.truncated_normal([hidden_layer_size, hidden_layer_size],
                                                  stddev = 1 / math.sqrt(float(image_size * image_size))))] * hidden_layer_num
  biases_hidden = [tf.Variable(tf.zeros([hidden_layer_size]))] * hidden_layer_num

  #output layer
  weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels],
                                                  stddev = 1 / math.sqrt(float(image_size * image_size))))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input) + biases_input)
  logits = tf.nn.dropout(logits, keep_prob)

  for i in range(hidden_layer_num):
    logits = tf.nn.relu(tf.matmul(logits, weights_hidden[i]) + biases_hidden[i])
    logits = tf.nn.dropout(logits, keep_prob)

  logits = tf.matmul(logits,  weights_output) + biases_output

  loss = tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
  loss += (regularization_beta * tf.nn.l2_loss(weights_input))
  for i in range(hidden_layer_num):
    loss += (regularization_beta * tf.nn.l2_loss(weights_hidden[i]))
  loss = tf.reduce_mean(loss)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_pre_tmp = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input)
  for i in range(hidden_layer_num):
    valid_pre_tmp = tf.nn.relu(tf.matmul(valid_pre_tmp, weights_hidden[i]) + biases_hidden[i])
  valid_prediction = tf.nn.softmax(tf.matmul(valid_pre_tmp, weights_output) + biases_output)

  test_pre_tmp = tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input)
  for i in range(hidden_layer_num):
    test_pre_tmp = tf.nn.relu(tf.matmul(test_pre_tmp, weights_hidden[i]) + biases_hidden[i])
  test_prediction = tf.nn.softmax(tf.matmul(test_pre_tmp, weights_output) + biases_output)


num_steps = 3001

start = time.clock()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : keep_probability}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print('%f sec' % (time.clock()- start))

Initialized
Minibatch loss at step 0: 3.813358
Minibatch accuracy: 10.2%
Validation accuracy: 29.0%
Minibatch loss at step 500: 1.141325
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 1.116450
Minibatch accuracy: 78.9%
Validation accuracy: 85.6%
Minibatch loss at step 1500: 0.947472
Minibatch accuracy: 80.5%
Validation accuracy: 85.4%
Minibatch loss at step 2000: 0.676430
Minibatch accuracy: 88.3%
Validation accuracy: 86.2%
Minibatch loss at step 2500: 0.757916
Minibatch accuracy: 85.9%
Validation accuracy: 86.5%
Minibatch loss at step 3000: 0.765083
Minibatch accuracy: 82.8%
Validation accuracy: 86.3%
Test accuracy: 92.5%
36.828070 sec


In [89]:
# set the offset to be 0 to restrict the trainning data
import math

hidden_layer_size = 1024
batch_size = 128
regularization_beta = 5e-4
keep_probability = 0.6

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  keep_prob = tf.placeholder(tf.float32)

  # Variables.
  # input layer
  weights_input = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size],
                                                  stddev = 1 / math.sqrt(float(image_size * image_size))))
  biases_input = tf.Variable(tf.zeros([hidden_layer_size]))

  # hidden layer 
  weights_hidden = tf.Variable(tf.truncated_normal([hidden_layer_size, hidden_layer_size], 
                                                   stddev = 1 / math.sqrt(float(image_size * image_size))))
  biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))

  #output layer
  weights_output = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels], 
                                                   stddev = 1 / math.sqrt(float(image_size * image_size))))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input) + biases_input)
  logits = tf.nn.dropout(logits, keep_prob)

  logits = tf.nn.relu(tf.matmul(logits, weights_hidden) + biases_hidden)
  logits = tf.nn.dropout(logits, keep_prob)

  logits = tf.matmul(logits,  weights_output) + biases_output

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) +
      regularization_beta * (tf.nn.l2_loss(weights_input) + tf.nn.l2_loss(weights_hidden) + tf.nn.l2_loss(weights_output)))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_pre_tmp = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input)
  valid_pre_tmp = tf.nn.relu(tf.matmul(valid_pre_tmp, weights_hidden) + biases_hidden)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_pre_tmp, weights_output) + biases_output)

  test_pre_tmp = tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input)
  test_pre_tmp = tf.nn.relu(tf.matmul(test_pre_tmp, weights_hidden) + biases_hidden)
  test_prediction = tf.nn.softmax(tf.matmul(test_pre_tmp, weights_output) + biases_output)


num_steps = 7001

start = time.clock()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : keep_probability}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
print('%f sec' % (time.clock()- start))

Initialized
Minibatch loss at step 0: 2.824532
Minibatch accuracy: 10.2%
Validation accuracy: 40.3%
Minibatch loss at step 500: 0.774764
Minibatch accuracy: 88.3%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.920636
Minibatch accuracy: 84.4%
Validation accuracy: 86.5%
Minibatch loss at step 1500: 0.891987
Minibatch accuracy: 82.8%
Validation accuracy: 87.0%
Minibatch loss at step 2000: 0.629967
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%
Minibatch loss at step 2500: 0.665131
Minibatch accuracy: 85.9%
Validation accuracy: 87.7%
Minibatch loss at step 3000: 0.637124
Minibatch accuracy: 84.4%
Validation accuracy: 87.9%
Minibatch loss at step 3500: 0.527360
Minibatch accuracy: 90.6%
Validation accuracy: 87.6%
Minibatch loss at step 4000: 0.636365
Minibatch accuracy: 85.9%
Validation accuracy: 88.7%
Minibatch loss at step 4500: 0.648263
Minibatch accuracy: 86.7%
Validation accuracy: 88.2%
Minibatch loss at step 5000: 0.594326
Minibatch accuracy: 87.5%
Validation accurac